In [ ]:
# This script loads T5 model and test and create slovenian paraphrases

In [ ]:
!pip install transformers
!pip install SentencePiece 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
from google.colab import drive

In [ ]:
# Set how many sentences to paraphrase and how many paraphrases for eaac sentence

numberOfSentences = 100
numberOfParaphrases = 10

drive.mount('/content/gdrive/', force_remount=True)
pathToModel = '/content/gdrive/MyDrive/NLPProject/paraphrasing/model/'

Mounted at /content/gdrive/


In [ ]:
# Reads sentences from Excel file
testSentences = pd.read_excel('/content/gdrive/MyDrive/NLPProject/paraphrasing/TestSentences.xlsx')
testSentences = testSentences.values.tolist()
testSentences = [i[0] for i in testSentences]
testSentences = testSentences[:numberOfSentences] 
print(len(testSentences))

100


In [ ]:
# Loads a model and tokeinzer

tokenizer = T5Tokenizer.from_pretrained("cjvt/t5-sl-small")
model = T5ForConditionalGeneration.from_pretrained(pathToModel)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32000, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32000, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): 

In [ ]:
outputTable = pd.DataFrame(index=range(len(testSentences)),columns=range(numberOfParaphrases+1))


for i in range(len(testSentences)):
    text =  "parafraza: " + testSentences[i]
    max_len = 256
    vrstica = [testSentences[i]]
    outputTable[0][i] = testSentences[i]
    encoding = tokenizer.encode_plus(text, padding='max_length', return_tensors="pt")
    input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)
    
    beam_outputs = model.generate(
        input_ids=input_ids, attention_mask=attention_masks,
        do_sample=True,
        max_length=256,
        top_k=120,
        top_p=0.98,
        early_stopping=True,
        num_return_sequences=numberOfParaphrases
    )
    

    print ("\nOriginal sentence:")
    print (testSentences[i])
    print ("\n")
    print ("Paraphrased sentences:")
    final_outputs =[]
    for beam_output in beam_outputs:
        sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
        if sent.lower() != testSentences[i].lower() and sent not in final_outputs:
            final_outputs.append(sent)
            
    for a, final_output in enumerate(final_outputs):
        print("{}: {}".format(a, final_output))
        outputTable[a+1][i] = final_output
        vrstica.append(final_output)
        
    print("\n-----------------------\n")

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.



Original sentence:
Vsaka občinska volilna komisija mora v občini določiti najmanj eno volišče, ki je dostopno invalidom.


Paraphrased sentences:
0: Vsaka občinska volilna komisija v občini mora določiti najmanj eno volišče, ki je brezplačno za invalide.
1: Vsaka občinska volilna komisija v občini mora določiti vsaj eno volišče, ki je brezplačno za invalide.

-----------------------


Original sentence:
Če nič ne delajo, ne morejo nič narediti narobe in jih nihče ne more pribijati na križ, pravi Simič.


Paraphrased sentences:
0: Če nič ne delajo, ne morejo ničesar narediti narobe in jih nihče ne more pribijati na križ, pravi Simič.
1: Če nič ne delajo, ne morejo nič narediti narobe, in jih nihče ne more pribijati na križ, pravi Simič.

-----------------------


Original sentence:
Po svoji naravi je svetovljanka in se znajde v katerikoli deželi, povsod je lahko doma.


Paraphrased sentences:
0: Po svojem značaju je svetovljan in je lahko doma.
1: Po svojem značaju je svetovljan in lah

In [ ]:
# Saving paraphrases to Excel faile

output = pd.DataFrame(np.column_stack([outputTable]), 
                               columns=["Original","Parafraza 1","Parafraza 2","Parafraza 3","Parafraza 4","Parafraza 5","Parafraza 6","Parafraza 7","Parafraza 8","Parafraza 9","Parafraza 10"])

output.to_excel("/content/gdrive/MyDrive/NLPProject/paraphrasing/Paraphrases.xlsx")